In [643]:
# Import all the libraries required

import soccerdata as sd
import pandas as pd
from tqdm import tqdm
import ScraperFC as sfc
import traceback

import warnings
import numpy as np
import socceraction.spadl as spadl

# from pyspark.sql import SparkSession
# from pyspark.sql.types import StructType, StructField, StringType
# import pyspark.pandas as ps


In [644]:
import datetime

In [645]:
## Create a connection to Snowflake
from sqlalchemy import create_engine
import snowflake.connector

In [646]:
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

In [647]:
pd.set_option('display.max_columns', None)

In [648]:
# spark = SparkSession.builder \
#     .master("local") \
#     .appName("Gegenstats") \
#     .getOrCreate()

In [649]:
scraper = sfc.Understat()
def scrape_league_data(years):
    output_df = pd.DataFrame(columns=['№', 'Team', 'M', 'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'NPxG', 'xGA',
       'NPxGA', 'NPxGD', 'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS', 'season','league'])
    for league_year in league_years:
        try:
            ooper = scraper.scrape_league_table(year=league_year[1], league=league_year[0])
            season_val = league_year[1]
            ooper['season'] = int(str(int(str(season_val)[2:])-1)+str(season_val)[2:])
            if league_year[0] == 'EPL':
                ooper['league'] = 'ENG-Premier League'
            output_df = pd.concat([output_df, ooper])
            
        except Exception as e:
            print(f"An error occurred for the year {league_years}: {e}")
            traceback.print_exc()
    scraper.close()
    return output_df

league_years = [('EPL', 2024)]  # Replace with your list of years
out = scrape_league_data(league_years)

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\ScraperFC\Understat.py:361: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  columns = pd.read_html(table)[0].columns
d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\ScraperFC\Understat.py:366: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table)[0]
C:\Users\ksbha\AppData\Local\Temp\ipykernel_6388\1491577790.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output_df = pd.concat([output_df, ooper])


In [650]:
understat_table = out[['league','Team','season', 'W', 'D', 'L','G','GA', 'xG', 'xGA','NPxG','NPxGA','PPDA',
                       'OPPDA','PTS','xPTS']]

In [651]:
understat_table.loc[understat_table['Team'] == 'Manchester United', 'Team'] = 'Manchester Utd'
understat_table.loc[understat_table['Team'] == 'Newcastle United', 'Team'] = 'Newcastle Utd'
understat_table.loc[understat_table['Team'] == 'Tottenham Hotspur', 'Team'] = 'Tottenham'
understat_table.loc[understat_table['Team'] == 'Wolverhampton Wanderers', 'Team'] = 'Wolves'
understat_table.loc[understat_table['Team'] == 'Nottingham Forest', 'Team'] = "Nott'ham Forest"
understat_table.loc[understat_table['Team'] == 'Leicester', 'Team'] = 'Leicester City'
understat_table.loc[understat_table['Team'] == 'Leeds', 'Team'] = 'Leeds United'
understat_table.loc[understat_table['Team'] == 'Sheffield United', 'Team'] = "Sheffield Utd"
understat_table.loc[understat_table['Team'] == 'Luton', 'Team'] = 'Luton Town'
understat_table.loc[understat_table['Team'] == 'Norwich', 'Team'] = 'Norwich City'

In [652]:
understat_table.sort_values(by=['league','season','Team'], ascending=True, inplace=True)
understat_table.reset_index(drop=True, inplace=True)

C:\Users\ksbha\AppData\Local\Temp\ipykernel_6388\614021855.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  understat_table.sort_values(by=['league','season','Team'], ascending=True, inplace=True)


In [653]:
understat_table.rename(columns={'Team':'TEAM_NAME', 'W':'TEAM_WINS', 'D':'TEAM_DRAWS', 'league':'COMPETITION',
                                'L': 'TEAM_LOSSES', 'xPTS':'TEAM_XPTS', 'PTS':'TEAM_PTS', 'season':'SEASON',
                                'NPxGA':'NPXG_AGAINST', 'xGA':'XG_AGAINST'}, inplace=True)

C:\Users\ksbha\AppData\Local\Temp\ipykernel_6388\2521636844.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  understat_table.rename(columns={'Team':'TEAM_NAME', 'W':'TEAM_WINS', 'D':'TEAM_DRAWS', 'league':'COMPETITION',


In [654]:
understat_table.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [641]:
fbref = sd.FBref(leagues="ENG-Premier League", seasons=["2324"], no_cache=True)
fbref_team_standard = fbref.read_team_season_stats(stat_type='standard')
# fbref_team_standard_oppo = fbref.read_team_season_stats(stat_type='standard', opponent_stats=True)
fbref_team_shooting = fbref.read_team_season_stats(stat_type='shooting')
fbref_team_shooting_oppo = fbref.read_team_season_stats(stat_type='shooting', opponent_stats=True)
fbref_team_passing = fbref.read_team_season_stats(stat_type='passing')
fbref_team_passing_oppo = fbref.read_team_season_stats(stat_type='passing', opponent_stats=True)
fbref_team_defense = fbref.read_team_season_stats(stat_type='defense')
fbref_team_possession = fbref.read_team_season_stats(stat_type='possession')
fbref_team_misc = fbref.read_team_season_stats(stat_type='misc')
fbref_keeper = fbref.read_team_season_stats(stat_type='keeper')
fbref_schedule = fbref.read_schedule()
fbref_lineups = fbref.read_lineup()

[02/02/24 23:56:47] INFO     Saving cached data to C:\Users\ksbha\soccerdata\data\FBref               ]8;id=826407;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=446783;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_common.py#91\91]8;;\

[02/02/24 23:58:36] INFO     [1/219] Retrieving game with id=3a6836b4                                  ]8;id=948354;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=174009;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#878\878]8;;\

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py:893: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_table.loc[:bench_idx, "is_starter"] = True
d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py:893: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_table.loc[:bench_idx, "is_starter"] = True


[02/02/24 23:58:40] INFO     [2/219] Retrieving game with id=26a7f90c                                  ]8;id=284403;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=269362;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#878\878]8;;\

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py:893: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_table.loc[:bench_idx, "is_starter"] = True
d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py:893: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_table.loc[:bench_idx, "is_starter"] = True


...

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py:893: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_table.loc[:bench_idx, "is_starter"] = True
d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py:893: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_table.loc[:bench_idx, "is_starter"] = True


[02/03/24 00:13:32] INFO     [218/219] Retrieving game with id=16dc9951                                ]8;id=221999;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=31336;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#878\878]8;;\

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py:893: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_table.loc[:bench_idx, "is_starter"] = True
d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py:893: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_table.loc[:bench_idx, "is_starter"] = True


[02/03/24 00:13:36] INFO     [219/219] Retrieving game with id=457203bc                                ]8;id=592356;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=502382;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#878\878]8;;\

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py:893: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_table.loc[:bench_idx, "is_starter"] = True
d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py:893: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_table.loc[:bench_idx, "is_starter"] = True


In [655]:
fbref_lineups.replace({"Jarrell Quansah": "Jarell Quansah"}, inplace=True)
fbref_lineups.replace({"Omotayo Adaramola": "Tayo Adaramola"}, inplace=True)

In [656]:
fbref_lineups.loc[fbref_lineups['team'] == 'Brighton & Hove Albion', 'team'] = 'Brighton'
fbref_lineups.loc[fbref_lineups['team'] == 'Manchester United', 'team'] = 'Manchester Utd'
fbref_lineups.loc[fbref_lineups['team'] == 'Newcastle United', 'team'] = 'Newcastle Utd'
fbref_lineups.loc[fbref_lineups['team'] == 'Nottingham Forest', 'team'] = "Nott'ham Forest"
fbref_lineups.loc[fbref_lineups['team'] == 'Sheffield United', 'team'] = "Sheffield Utd"
fbref_lineups.loc[fbref_lineups['team'] == 'Tottenham Hotspur', 'team'] = 'Tottenham'
fbref_lineups.loc[fbref_lineups['team'] == 'West Ham United', 'team'] = 'West Ham'
fbref_lineups.loc[fbref_lineups['team'] == 'Wolverhampton Wanderers', 'team'] = 'Wolves'

In [657]:
fbref_team_standard = fbref_team_standard.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
# fbref_team_standard_oppo = fbref_team_standard_oppo.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_shooting = fbref_team_shooting.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_shooting_oppo = fbref_team_shooting_oppo.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_passing = fbref_team_passing.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_passing_oppo = fbref_team_passing_oppo.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_defense = fbref_team_defense.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_possession = fbref_team_possession.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_misc = fbref_team_misc.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_keeper = fbref_keeper.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_schedule = fbref_schedule.rename_axis(index={'season': 'SEASON', 'league': 'COMPETITION'})
fbref_lineups = fbref_lineups.rename_axis(index={'season': 'SEASON', 'league': 'COMPETITION'})
fbref_lineups.rename({'team': 'TEAM_NAME'}, axis=1, inplace=True)

In [658]:
## Get rid of incomplete games (Thank God Tom Lockyear is good <3)
fbref_schedule = fbref_schedule.dropna(subset=['home_xg'])

## Get rid of 'vs ' in the TEAM_NAME column
fbref_team_passing_oppo.reset_index(inplace=True)
fbref_team_shooting_oppo.reset_index(inplace=True)

In [659]:
fbref_team_passing_oppo['TEAM_NAME'] = fbref_team_passing_oppo['TEAM_NAME'].str.replace('vs ', '')
fbref_team_shooting_oppo['TEAM_NAME'] = fbref_team_shooting_oppo['TEAM_NAME'].str.replace('vs ', '')

fbref_team_passing_oppo.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)
fbref_team_shooting_oppo.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [660]:
# Convert the date column to datetime format
fbref_schedule['date'] = pd.to_datetime(fbref_schedule['date'])
# fbref_schedule = fbref_schedule[fbref_schedule['date'] < datetime.datetime(2023, 12, 20)]

fbref_schedule = fbref_schedule[fbref_schedule['date'] <= datetime.datetime.now().strftime('%Y-%m-%d')]

In [661]:
def make_season_integer(df):
    indexes = df.index.names
    df = df.reset_index()
    df['SEASON'] = df['SEASON'].astype(int)
    df = df.set_index(indexes)
    return df

fbref_team_standard = make_season_integer(fbref_team_standard)
# fbref_team_standard_oppo = make_season_integer(fbref_team_standard_oppo)
fbref_team_shooting = make_season_integer(fbref_team_shooting)
fbref_team_shooting_oppo = make_season_integer(fbref_team_shooting_oppo)
fbref_team_passing = make_season_integer(fbref_team_passing)
fbref_team_passing_oppo = make_season_integer(fbref_team_passing_oppo)
fbref_team_defense = make_season_integer(fbref_team_defense)
fbref_team_possession = make_season_integer(fbref_team_possession)
fbref_team_misc = make_season_integer(fbref_team_misc)
fbref_schedule = make_season_integer(fbref_schedule)
fbref_keeper = make_season_integer(fbref_keeper)
fbref_lineups = make_season_integer(fbref_lineups)

In [662]:
team_matches_played = fbref_team_standard['Playing Time'].MP
team_goals_scored = understat_table.G
team_npxG_90 = fbref_team_standard['Per 90 Minutes'].npxG
team_goals_conceded = understat_table.GA
# team_xG_against_90 = fbref_team_standard_oppo['Per 90 Minutes'].xG
team_shots = fbref_team_shooting['Standard'].Sh
team_shotsOT = fbref_team_shooting['Standard'].SoT
team_pass_completed = fbref_team_passing["Total"].Cmp
team_pass_attempted = fbref_team_passing["Total"].Att
team_tackles = fbref_team_defense.Tackles.Tkl
team_tackles_won = fbref_team_defense.Tackles.TklW
team_takeons_attempted = fbref_team_possession['Take-Ons'].Att
team_takeons_completed = fbref_team_possession['Take-Ons'].Succ
team_crossesintoPA = fbref_team_passing['CrsPA']
team_fouls_against = fbref_team_misc['Performance']['Fld']
team_fouls_made = fbref_team_misc['Performance']['Fls']
team_interceptions = fbref_team_defense['Int']
team_blocks_shots = fbref_team_defense['Blocks'].Sh
team_blocks_pass = fbref_team_defense['Blocks'].Pass
team_clearances = fbref_team_defense.Clr
team_aerials_won = fbref_team_misc['Aerial Duels'].Won
team_aerials_lost = fbref_team_misc['Aerial Duels'].Lost
team_shots_faced = fbref_team_shooting_oppo['Standard'].Sh

In [663]:
team_recoveries= fbref_team_misc['Performance']['Recov']

In [664]:
teams_poss_won = team_interceptions + team_tackles_won + team_recoveries + team_blocks_shots + \
    team_blocks_pass + team_aerials_won
team_OPPDA = understat_table['OPPDA']
team_final_3rd_passes_against =  fbref_team_passing_oppo['1/3']
team_cleansheets = fbref_keeper['Performance']['CS']

In [665]:
team_names = fbref_team_standard.reset_index()[['TEAM_NAME','url']]
team_names['TEAM_FBREF_ID'] = team_names['url'].apply(lambda x: x.split('/')[3])      ## Extracting team_id from URL
team_names.drop(columns=['url'], inplace=True)
team_names.columns = team_names.columns.droplevel(1)

C:\Users\ksbha\AppData\Local\Temp\ipykernel_6388\2026419047.py:3: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  team_names.drop(columns=['url'], inplace=True)


In [666]:
team_names['TEAM_FBREF_ID'] = team_names['TEAM_FBREF_ID'].astype('string')

In [667]:
team_names.drop_duplicates(inplace=True)

## WhoScored Data

In [668]:
ws = sd.WhoScored(leagues='ENG-Premier League', seasons=["2324"])

[02/03/24 00:16:12] INFO     Saving cached data to C:\Users\ksbha\soccerdata\data\WhoScored           ]8;id=698677;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=315697;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_common.py#91\91]8;;\

In [669]:
ws_schedule = ws.read_schedule()

[02/03/24 00:16:25] INFO     Scraping game schedule from                                           ]8;id=912137;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=995738;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#427\427]8;;\
                             https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/9618/Engl                 
                             and-Premier-League                                                                    

                    INFO     Scraping game schedule for Thursday, Feb 1 2024                       ]8;id=230701;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=718386;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

                    INFO     Scraping game schedule for Saturday, Feb 3 2024                       ]8;id=199277;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=588204;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

[02/03/24 00:16:26] INFO     Scraping game schedule for Sunday, Feb 4 2024                         ]8;id=2144;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=503777;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

                    INFO     Scraping game schedule for Monday, Feb 5 2024                         ]8;id=494914;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=344549;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

...

                    INFO     Scraping game schedule for Sunday, Aug 20 2023                        ]8;id=476115;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=43005;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

[02/03/24 00:17:33] INFO     Scraping game schedule for Monday, Aug 21 2023                        ]8;id=113352;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=988408;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

                    INFO     Scraping game schedule for Friday, Aug 25 2023                        ]8;id=424018;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=359734;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

                    INFO     Scraping game schedule for Saturday, Aug 26 2023                      ]8;id=77677;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=300948;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

[02/03/24 00:17:34] INFO     Scraping game schedule for Sunday, Aug 27 2023                        ]8;id=393851;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=275166;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

In [670]:
ws_schedule = ws_schedule[ws_schedule['date'] <= datetime.datetime.now().strftime('%Y-%m-%d')]

In [671]:
fbref_schedule = fbref_schedule.dropna(subset=['home_xg'])

In [672]:
ws_schedule = ws_schedule.reset_index().rename({'league':"COMPETITION", "season":"SEASON"}, axis=1)
ws_schedule.set_index(['COMPETITION', 'SEASON', 'game'], inplace=True)

In [673]:
ws_schedule.loc[ws_schedule['home_team'] == 'Manchester United', 'home_team'] = 'Manchester Utd'
ws_schedule.loc[ws_schedule['away_team'] == 'Manchester United', 'away_team'] = 'Manchester Utd'
ws_schedule.loc[ws_schedule['home_team'] == 'Newcastle United', 'home_team'] = 'Newcastle Utd'
ws_schedule.loc[ws_schedule['away_team'] == 'Newcastle United', 'away_team'] = 'Newcastle Utd'
ws_schedule.loc[ws_schedule['home_team'] == 'Newcastle', 'home_team'] = 'Newcastle Utd'
ws_schedule.loc[ws_schedule['away_team'] == 'Newcastle', 'away_team'] = 'Newcastle Utd'
ws_schedule.loc[ws_schedule['home_team'] == 'Tottenham Hotspur', 'home_team'] = 'Tottenham'
ws_schedule.loc[ws_schedule['away_team'] == 'Tottenham Hotspur', 'away_team'] = 'Tottenham'
ws_schedule.loc[ws_schedule['home_team'] == 'Wolverhampton Wanderers', 'home_team'] = 'Wolves'
ws_schedule.loc[ws_schedule['away_team'] == 'Wolverhampton Wanderers', 'away_team'] = 'Wolves'
ws_schedule.loc[ws_schedule['home_team'] == 'Wolverhampton', 'home_team'] = 'Wolves'
ws_schedule.loc[ws_schedule['away_team'] == 'Wolverhampton', 'away_team'] = 'Wolves'
ws_schedule.loc[ws_schedule['home_team'] == 'Nottingham Forest', 'home_team'] = "Nott'ham Forest"
ws_schedule.loc[ws_schedule['away_team'] == 'Nottingham Forest', 'away_team'] = "Nott'ham Forest"
ws_schedule.loc[ws_schedule['home_team'] == 'Leicester', 'home_team'] = "Leicester City"
ws_schedule.loc[ws_schedule['away_team'] == 'Leicester', 'away_team'] = "Leicester City"
ws_schedule.loc[ws_schedule['home_team'] == 'Leeds', 'home_team'] = "Leeds United"
ws_schedule.loc[ws_schedule['away_team'] == 'Leeds', 'away_team'] = "Leeds United"
ws_schedule.loc[ws_schedule['home_team'] == 'Sheffield United', 'home_team'] = "Sheffield Utd"
ws_schedule.loc[ws_schedule['away_team'] == 'Sheffield United', 'away_team'] = "Sheffield Utd"
ws_schedule.loc[ws_schedule['home_team'] == 'Luton', 'home_team'] = "Luton Town"
ws_schedule.loc[ws_schedule['away_team'] == 'Luton', 'away_team'] = "Luton Town"
ws_schedule.loc[ws_schedule['home_team'] == 'Norwich', 'home_team'] = "Norwich City"
ws_schedule.loc[ws_schedule['away_team'] == 'Norwich', 'away_team'] = "Norwich City"

In [674]:
ws_schedule.reset_index(inplace=True)
fbref_schedule.reset_index(inplace=True)

In [675]:
ws_schedule = ws_schedule.rename({'game_id':"ws_game_id"}, axis=1)

In [676]:
ws_schedule['SEASON'] = ws_schedule['SEASON'].astype('int64')

In [766]:
fbref_schedule = pd.merge(fbref_schedule, ws_schedule[["COMPETITION", "SEASON", "home_team", "away_team","ws_game_id"]], 
          on=["COMPETITION", "SEASON", "home_team", "away_team"], how='inner')

In [38]:
loader = ws.read_events(output_fmt='loader')

[02/02/24 17:33:43] INFO     Scraping game schedule from                                           ]8;id=919188;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=99265;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#427\427]8;;\
                             https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/9618/Engl                 
                             and-Premier-League                                                                    

                    INFO     Scraping game schedule for Thursday, Feb 1 2024                       ]8;id=97089;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=272090;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

                    INFO     Scraping game schedule for Saturday, Feb 3 2024                       ]8;id=892953;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=399012;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

                    INFO     Scraping game schedule for Sunday, Feb 4 2024                         ]8;id=135550;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=994491;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

[02/02/24 17:33:44] INFO     Scraping game schedule for Monday, Feb 5 2024                         ]8;id=451210;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=442990;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

...

[02/02/24 17:47:30] INFO     [256/259] Retrieving game with id=1729305                             ]8;id=274657;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=501306;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#693\693]8;;\

                    INFO     [257/259] Retrieving game with id=1729213                             ]8;id=222894;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=8569;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#693\693]8;;\

                    INFO     [258/259] Retrieving game with id=1729477                             ]8;id=140216;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=589267;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#693\693]8;;\

[02/02/24 17:47:47] WARNING  No events found for game 1729477                                      ]8;id=45363;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=431500;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#757\757]8;;\

                    INFO     [259/259] Retrieving game with id=1729494                             ]8;id=535577;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=951935;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#693\693]8;;\

In [271]:
game_ids = ws_schedule['ws_game_id'].unique()

In [272]:
ws_players = pd.DataFrame(columns=['game_id', 'team_id', 'player_id', 'player_name', 
                                   'is_starter','minutes_played', 'jersey_number', 'starting_position'])

In [273]:
for game in tqdm(game_ids):
    ws_players = pd.concat([ws_players, loader.players(game_id=game)])

100%|██████████| 218/218 [00:10<00:00, 20.94it/s]


In [274]:
ws_players.head()

,game_id,team_id,player_id,player_name,is_starter,minutes_played,jersey_number,starting_position
0,1729191,184,404852,James Trafford,True,104,1,GK
1,1729191,184,135865,Connor Roberts,True,104,14,DR
2,1729191,184,419917,Ameen Al Dakhil,True,104,28,DC
3,1729191,184,353412,Jordan Beyer,True,79,5,DC
4,1729191,184,362826,Dara O'Shea,True,104,2,DC


In [275]:
## WhoScored decide to name Willy Boly with two different names. Idiots
ws_players.replace({"Willy-Arnaud Boly": "Willy Boly"}, inplace=True)
ws_players.replace({"Kepa": "Kepa Arrizabalaga"}, inplace=True)
ws_players.replace({"Yehor Yarmolyuk": "Yehor Yarmoliuk"}, inplace=True)
ws_players.replace({"Vitalii Mykolenko": "Vitaliy Mykolenko"}, inplace=True)
ws_players.replace({"Dan Bentley": "Daniel Bentley"}, inplace=True)
ws_players.replace({"Toti": "Toti Gomes"}, inplace=True)
ws_players.replace({"Joe Hodge": "Joseph Hodge"}, inplace=True)
ws_players.replace({"Jonny Otto": "Jonny"}, inplace=True)
ws_players.replace({"Álvaro Carreras": "Álvaro Fernández"}, inplace=True)
ws_players.replace({"Alisson Becker": "Alisson"}, inplace=True)
ws_players.replace({"Andy Robertson": "Andrew Robertson"}, inplace=True)
ws_players.replace({"Konstantinos Tsimikas": "Kostas Tsimikas"}, inplace=True)
ws_players.replace({"Alphonse Aréola": "Alphonse Areola"}, inplace=True)
ws_players.replace({"Pape Sarr": "Pape Matar Sarr"}, inplace=True)
ws_players.replace({"Yago Alonso": "Yago Santiago"}, inplace=True)

## Snowflake Data Upload

In [678]:
SNOWFLAKE_USER = 'kbharaj3'
SNOWFLAKE_PASSWORD = 'Snowfl@key0014'
SNOWFLAKE_ACCOUNT = 'qx25653.ca-central-1.aws'
SNOWFLAKE_WAREHOUSE = 'FOOTY_STORE'
SNOWFLAKE_DATABASE = 'GEGENSTATS'
SNOWFLAKE_SCHEMA = 'TABLES'

In [679]:
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
    )

[02/03/24 00:17:56] INFO     Snowflake Connector for Python Version: 3.3.1, Python Version:       ]8;id=506361;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py\connection.py]8;;\:]8;id=848873;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py#334\334]8;;\
                             3.11.5, Platform: Windows-10-10.0.22000-SP0                                           

                    INFO     This connection is in OCSP Fail Open Mode. TLS Certificates would   ]8;id=471155;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py\connection.py]8;;\:]8;id=238950;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py#1103\1103]8;;\
                             be checked for validity and revocation status. Any other                              
                             Certificate Revocation related exceptions or OCSP Responder                           
                             failures would be disregarded in favor of connectivity.                               

In [680]:
cur = conn.cursor()
cur.execute(f"USE WAREHOUSE {SNOWFLAKE_WAREHOUSE}")

[02/03/24 00:17:58] INFO     query: [USE WAREHOUSE FOOTY_STORE]                                       ]8;id=109594;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=679840;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=677878;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=109292;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=675584;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=338836;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [681]:
create_schema_sql = f"CREATE SCHEMA IF NOT EXISTS {SNOWFLAKE_SCHEMA}"
cur.execute(create_schema_sql)

[02/03/24 00:17:59] INFO     query: [CREATE SCHEMA IF NOT EXISTS TABLES]                              ]8;id=788348;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=822735;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=850411;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=148845;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=710740;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=954820;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [682]:
print("The Snowflake warehouse being used is:", SNOWFLAKE_WAREHOUSE)

The Snowflake warehouse being used is: FOOTY_STORE


In [683]:
cur.execute('SELECT * FROM TEAMS')
team_rows = cur.fetchall()
column_names = [desc[0] for desc in cur.description]
curr_team_names = pd.DataFrame(team_rows, columns=column_names)

[02/03/24 00:18:06] INFO     query: [SELECT * FROM TEAMS]                                             ]8;id=613625;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=241361;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=443409;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=900698;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 25                                     ]8;id=229901;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=778577;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [684]:
## Below (if not blank) are the teams you need to create LOGOS for
team_names_not_in_curr = team_names[~team_names['TEAM_NAME'].isin(curr_team_names['TEAM_NAME'])]
team_names_not_in_curr

,TEAM_NAME,TEAM_FBREF_ID


In [685]:
def upsert_to_snowflake(table_name, dataframe, primary_keys):
    if isinstance(primary_keys, str):
        primary_keys = [primary_keys]

    temp_table_name = f"{table_name}_TEMP"
    success, nchunks, nrows, _ = write_pandas(conn, dataframe, temp_table_name, auto_create_table=True)

    on_condition = ' AND '.join([f"{table_name}.{pk} = {temp_table_name}.{pk}" for pk in primary_keys])

    update_columns = [col for col in dataframe.columns if col not in primary_keys]
    update_sql = ', '.join([f"{table_name}.{col} = {temp_table_name}.{col}" for col in update_columns])

    merge_sql = f"""
    MERGE INTO {table_name} USING {temp_table_name}
    ON {on_condition}
    """

    if update_columns:
        merge_sql += f"""
        WHEN MATCHED THEN
            UPDATE SET
                {update_sql}
        """

    merge_sql += f"""
    WHEN NOT MATCHED THEN
        INSERT ({', '.join(dataframe.columns)})
        VALUES ({', '.join([f"{temp_table_name}.{col}" for col in dataframe.columns])})
    """

    cur.execute(merge_sql)
    cur.execute(f"DROP TABLE IF EXISTS {temp_table_name}")

    print(f"Upsert completed. {nrows} rows processed.")

In [686]:
upsert_to_snowflake("TEAMS", team_names, ['TEAM_FBREF_ID'])

[02/03/24 00:18:19] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=822133;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=908299;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

[02/03/24 00:18:20] INFO     query execution done                                                     ]8;id=57426;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=68628;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=758862;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=456345;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=884545;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=521933;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=909898;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=855452;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

[02/03/24 00:18:23] INFO     query execution done                                                     ]8;id=183126;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=538207;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS TEAMS_TEMP]                                 ]8;id=618959;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=675867;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=660796;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=505374;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=295694;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=374323;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 20 rows processed.


In [687]:
cur.execute('SELECT * FROM TEAMS')
team_rows = cur.fetchall()
column_names = [desc[0] for desc in cur.description]
team_names = pd.DataFrame(team_rows, columns=column_names)

[02/03/24 00:18:30] INFO     query: [SELECT * FROM TEAMS]                                             ]8;id=703166;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=185189;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=872635;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=523719;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 25                                     ]8;id=997721;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=597845;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [688]:
team_names.drop(columns=['TEAM_LOGO_URL'], inplace=True)

In [689]:
team_standard_stats = understat_table.copy()

In [691]:
# team_standard_stats.reset_index(inplace=True)
# team_standard_stats.loc[team_standard_stats['TEAM_NAME'] == 'Luton', 'TEAM_NAME'] = 'Luton Town'
# team_standard_stats.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [692]:
team_standard_stats['MATCHES_PLAYED'] = team_matches_played

In [693]:
team_standard_stats.reset_index(inplace=True)

In [694]:
team_standard_stats = team_standard_stats.merge(team_names, on='TEAM_NAME', how='left')

In [695]:
team_standard_stats = team_standard_stats[['TEAM_FBREF_ID','SEASON','COMPETITION','MATCHES_PLAYED','TEAM_WINS', 
                                           'TEAM_DRAWS', 'TEAM_LOSSES', 'TEAM_PTS', 'TEAM_XPTS']]

In [696]:
upsert_to_snowflake("TEAM_STANDARD_STATS", team_standard_stats, ['TEAM_FBREF_ID', 'COMPETITION','SEASON'])

[02/03/24 00:19:16] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=279420;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=203686;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=394864;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=453048;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=375602;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=349568;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=170994;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=691432;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=580147;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=444958;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

[02/03/24 00:19:19] INFO     query execution done                                                     ]8;id=49804;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=480906;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS TEAM_STANDARD_STATS_TEMP]                   ]8;id=576253;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=757512;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=107149;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=535016;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=183781;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=879554;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 20 rows processed.


In [697]:
team_attacking_stats = understat_table.copy()

In [698]:
team_attacking_stats = team_attacking_stats.reset_index().merge(team_names, on='TEAM_NAME', how='left')

In [699]:
team_attacking_stats = team_attacking_stats[['TEAM_NAME','TEAM_FBREF_ID','COMPETITION','SEASON','NPxG','xG']]

In [700]:
team_attacking_stats.rename(columns={'NPxG': 'NPXG', 'xG':'XG'}, inplace=True)

In [701]:
team_attacking_stats.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [702]:
team_attacking_stats['GOALS_SCORED'] = team_goals_scored
team_attacking_stats['SHOTS'] = team_shots
team_attacking_stats['SHOTS_ON_TARGET'] = team_shotsOT
team_attacking_stats['PASS_COMPLETED'] = team_pass_completed
team_attacking_stats['PASS_ATTEMPTED'] = team_pass_attempted
team_attacking_stats['TAKEONS_ATTEMPTED'] = team_takeons_attempted
team_attacking_stats['TAKEONS_COMPLETED'] = team_takeons_completed
team_attacking_stats['CROSSES_INTO_PA'] = team_crossesintoPA
team_attacking_stats['FOULS_AGAINST'] = team_fouls_against

In [703]:
team_attacking_stats.reset_index(inplace=True)
team_attacking_stats.drop(columns=['TEAM_NAME'], inplace=True)

In [704]:
upsert_to_snowflake("TEAM_ATTACKING_STATS", team_attacking_stats, ['TEAM_FBREF_ID', 'COMPETITION','SEASON'])

[02/03/24 00:20:15] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=520761;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=964390;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=656258;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=479964;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=609239;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=896602;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=793946;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=490979;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

[02/03/24 00:20:16] INFO     query execution done                                                     ]8;id=605245;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=331015;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

                    INFO     query execution done                                                     ]8;id=97469;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=906216;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS TEAM_ATTACKING_STATS_TEMP]                  ]8;id=369724;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=162919;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=698831;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=444616;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=651665;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=893713;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 20 rows processed.


In [705]:
team_defending_stats = understat_table.copy()

In [706]:
team_defending_stats = team_defending_stats.reset_index().merge(team_names, on='TEAM_NAME', how='left')

In [707]:
team_defending_stats = team_defending_stats[['TEAM_NAME','TEAM_FBREF_ID','COMPETITION','SEASON']]

In [708]:
team_defending_stats.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [709]:
team_defending_stats['GOALS_CONCEDED'] = team_goals_conceded
team_defending_stats['XG_AGAINST'] = understat_table['XG_AGAINST']
team_defending_stats['NPXG_AGAINST'] = understat_table['NPXG_AGAINST']
team_defending_stats['TACKLES'] = team_tackles
team_defending_stats['TACKLES_WON'] = team_tackles_won
team_defending_stats['FOULS_MADE'] = team_fouls_made
team_defending_stats['INTERCEPTIONS'] = team_interceptions
team_defending_stats['BLOCKED_SHOTS'] = team_blocks_shots
team_defending_stats['BLOCKED_PASSES'] = team_blocks_pass
team_defending_stats['CLEARANCES'] = team_clearances
team_defending_stats['PPDA'] = understat_table['PPDA']
team_defending_stats['OPP_PPDA'] = team_OPPDA
team_defending_stats['POSS_WON'] = teams_poss_won
team_defending_stats['FINAL_3RD_PASSES_AGAINST'] = team_final_3rd_passes_against
team_defending_stats['CLEAN_SHEETS'] = team_cleansheets
team_defending_stats['SHOTS_FACED'] = team_shots_faced

In [710]:
team_defending_stats.reset_index(inplace=True)
team_defending_stats.drop(columns=['TEAM_NAME'], inplace=True)

In [711]:
# sql_command = """
# ALTER TABLE GEGENSTATS.TABLES.TEAM_DEFENDING_STATS 
# ADD COLUMN SHOTS_FACED NUMBER(38,0);
# """

# cur.execute(sql_command)

In [712]:
# team_shots_faced = team_shots_faced.reset_index()

In [713]:
# for index, row in team_shots_faced.iterrows():
#     sql_command = """
#     UPDATE GEGENSTATS.TABLES.TEAM_DEFENDING_STATS
#     SET SHOTS_FACED = %s
#     WHERE TEAM_FBREF_ID = (SELECT TEAM_FBREF_ID FROM GEGENSTATS.TABLES.TEAMS WHERE TEAM_NAME = %s) 
#       AND COMPETITION = %s
#       AND SEASON = %s
#     """

#     values = (row['Sh'], row['TEAM_NAME'], row['COMPETITION'], row['SEASON'])

#     cur.execute(sql_command, values)

In [714]:
upsert_to_snowflake("TEAM_DEFENDING_STATS", team_defending_stats, ['TEAM_FBREF_ID', 'COMPETITION','SEASON'])

[02/03/24 00:21:44] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=960922;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=326554;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=855828;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=952539;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=166855;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=792831;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=465557;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=982180;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

[02/03/24 00:21:45] INFO     query execution done                                                     ]8;id=759463;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=896370;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

[02/03/24 00:21:48] INFO     query execution done                                                     ]8;id=820040;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=89246;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS TEAM_DEFENDING_STATS_TEMP]                  ]8;id=720660;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=31679;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=37847;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=172202;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=290107;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=538328;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 20 rows processed.


In [715]:
team_misc_stats = understat_table.copy()

In [716]:
team_misc_stats = team_misc_stats.reset_index().merge(team_names, on='TEAM_NAME', how='left')

In [717]:
team_misc_stats = team_misc_stats[['TEAM_NAME','TEAM_FBREF_ID','COMPETITION','SEASON']]

In [718]:
team_misc_stats.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [719]:
team_misc_stats['AERIALS_WON'] = team_aerials_won
team_misc_stats['AERIALS_LOST'] = team_aerials_lost

In [720]:
team_misc_stats.reset_index(inplace=True)
team_misc_stats.drop(columns=['TEAM_NAME'], inplace=True)

In [721]:
upsert_to_snowflake("TEAM_MISC_STATS", team_misc_stats, ['TEAM_FBREF_ID', 'COMPETITION','SEASON'])

[02/03/24 00:21:57] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=556112;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=867610;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=257255;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=128233;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=742745;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=903875;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=256865;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=452528;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=488767;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=842098;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

[02/03/24 00:22:00] INFO     query execution done                                                     ]8;id=669672;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=912152;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS TEAM_MISC_STATS_TEMP]                       ]8;id=377823;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=627574;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=479683;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=741244;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=128122;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=380508;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 20 rows processed.


## Competition, Season and Match Data

In [722]:
df_competitions = fbref_schedule.reset_index()[['COMPETITION', 'SEASON']].drop_duplicates().copy()

In [723]:
df_competitions['COMPETITION_NAME'] = ''
df_competitions['COMPETITION_ACRONYM'] = ''

In [724]:
df_competitions.loc[df_competitions['COMPETITION'] == 'ENG-Premier League', 'COMPETITION_NAME'] = 'English Premier League'
df_competitions.loc[df_competitions['COMPETITION'] == 'ENG-Premier League', 'COMPETITION_ACRONYM'] = 'EPL'

In [725]:
df_competitions

,COMPETITION,SEASON,COMPETITION_NAME,COMPETITION_ACRONYM
0,ENG-Premier League,2324,English Premier League,EPL


In [726]:
upsert_to_snowflake("COMPETITIONS", df_competitions, ['COMPETITION','SEASON'])

C:\Users\ksbha\AppData\Local\Temp\ipykernel_6388\138046206.py:6: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, nchunks, nrows, _ = write_pandas(conn, dataframe, temp_table_name, auto_create_table=True)


[02/03/24 00:22:14] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=584206;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=6003;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=707377;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=627592;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=666461;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=652715;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=213671;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=749441;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

...

[02/03/24 00:22:17] INFO     query execution done                                                     ]8;id=578402;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=339994;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS COMPETITIONS_TEMP]                          ]8;id=805650;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=948061;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=832907;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=692539;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=75225;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=643447;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 1 rows processed.


In [727]:
df_seasons = df_competitions[['SEASON']].drop_duplicates().copy()

In [728]:
df_seasons['SEASON_NAME'] = ''
df_seasons['START_YEAR'] = 0
df_seasons['END_YEAR'] = 0

In [729]:
# df_seasons.loc[df_seasons['SEASON'] == 2122, 'SEASON_NAME'] = '2021-2022'
# df_seasons.loc[df_seasons['SEASON'] == 2122, 'START_YEAR'] = 2021
# df_seasons.loc[df_seasons['SEASON'] == 2122, 'END_YEAR'] = 2022
df_seasons.loc[df_seasons['SEASON'] == 2324, 'SEASON_NAME'] = '2023-2024'
df_seasons.loc[df_seasons['SEASON'] == 2324, 'START_YEAR'] = 2023
df_seasons.loc[df_seasons['SEASON'] == 2324, 'END_YEAR'] = 2024

In [730]:
df_seasons

,SEASON,SEASON_NAME,START_YEAR,END_YEAR
0,2324,2023-2024,2023,2024


In [731]:
upsert_to_snowflake("SEASONS", df_seasons, ['SEASON'])

C:\Users\ksbha\AppData\Local\Temp\ipykernel_6388\138046206.py:6: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, nchunks, nrows, _ = write_pandas(conn, dataframe, temp_table_name, auto_create_table=True)


[02/03/24 00:22:36] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=960012;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=24236;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=187255;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=848075;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=453456;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=544985;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=987055;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=480663;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

...

                    INFO     query execution done                                                     ]8;id=755598;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=990214;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS SEASONS_TEMP]                               ]8;id=149644;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=558371;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=608184;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=709220;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=183155;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=359641;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 1 rows processed.


In [732]:
df_stadiums = fbref_schedule.reset_index()[['home_team', 'SEASON',
            'venue']].drop_duplicates().rename(columns={'home_team': 'TEAM_NAME', 'venue': 'STADIUM'})

In [733]:
df_stadiums = df_stadiums.merge(team_names, on='TEAM_NAME')[['STADIUM','SEASON','TEAM_FBREF_ID']]

In [734]:
upsert_to_snowflake("STADIUMS", df_stadiums, ['TEAM_FBREF_ID','STADIUM', 'SEASON'])

[02/03/24 00:22:44] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=85259;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=122533;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=817435;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=453181;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=308517;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=894083;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=284831;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=166754;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

[02/03/24 00:22:45] INFO     query execution done                                                     ]8;id=353136;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=996662;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

                    INFO     query execution done                                                     ]8;id=128387;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=88040;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS STADIUMS_TEMP]                              ]8;id=188422;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=495223;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=764137;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=866612;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=307310;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=111208;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 20 rows processed.


In [788]:
df_matches = fbref_schedule.reset_index()[['game_id', 'date', 'time', 'home_team', 'away_team', 'COMPETITION', 'SEASON', 'venue',
                                           'week', 'day', 'score', 'home_xg', 'away_xg', 'attendance', 'referee', 'ws_game_id']]

In [792]:
df_matches

,game_id,date,time,home_team,away_team,COMPETITION,SEASON,venue,week,day,score,home_xg,away_xg,attendance,referee,ws_game_id,home_team_id,away_team_id
0,3a6836b4,2023-08-11,20:00,Burnley,Manchester City,ENG-Premier League,2324,Turf Moor,1,Fri,0–3,0.3,1.9,21572,Craig Pawson,1729191,943e8050,b8fd03ef
1,26a7f90c,2023-08-12,12:30,Arsenal,Nott'ham Forest,ENG-Premier League,2324,Emirates Stadium,1,Sat,2–1,0.8,1.2,59984,Michael Oliver,1729190,18bb7c10,e4a775cb
2,d6bbf293,2023-08-12,15:00,Bournemouth,West Ham,ENG-Premier League,2324,Vitality Stadium,1,Sat,1–1,1.3,1.1,11245,Peter Bankes,1729192,4ba7cbea,7c21e445
3,56a137f7,2023-08-12,15:00,Brighton,Luton Town,ENG-Premier League,2324,The American Express Community Stadium,1,Sat,4–1,4.0,1.5,31872,David Coote,1729193,d07537b9,e297cd13
4,15addfc7,2023-08-12,15:00,Everton,Fulham,ENG-Premier League,2324,Goodison Park,1,Sat,0–1,2.7,1.5,39940,Stuart Attwell,1729194,d3fd31cc,fd962109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,1836d9bd,2024-01-31,20:15,Liverpool,Chelsea,ENG-Premier League,2324,Anfield,22,Wed,4–1,2.9,0.6,<NA>,Paul Tierney,1729459,822bd0ba,cff3d9bb
214,d10aaae8,2024-01-31,19:30,Manchester City,Burnley,ENG-Premier League,2324,Etihad Stadium,22,Wed,3–1,1.7,1.4,<NA>,Samuel Barrott,1729460,b8fd03ef,943e8050
215,79ffec97,2024-01-31,19:30,Tottenham,Brentford,ENG-Premier League,2324,Tottenham Hotspur Stadium,22,Wed,3–2,2.9,1.6,61210,David Coote,1729455,361ca564,cd051869
216,16dc9951,2024-02-01,19:30,West Ham,Bournemouth,ENG-Premier League,2324,London Stadium,22,Thu,1–1,1.4,1.2,<NA>,Tim Robinson,1729456,7c21e445,4ba7cbea


In [790]:
df_matches = df_matches.merge(team_names[['TEAM_FBREF_ID','TEAM_NAME']].rename(columns={'TEAM_FBREF_ID': 'home_team_id', 
                                                         'TEAM_NAME': 'home_team'}), on='home_team', how='left')

In [791]:
df_matches = df_matches.merge(team_names[['TEAM_FBREF_ID','TEAM_NAME']].rename(columns={'TEAM_FBREF_ID': 'away_team_id', 
                                                         'TEAM_NAME': 'away_team'}), on='away_team', how='left')

In [793]:
# Combine df_matches date and time to a dt.datatime configuration
df_matches['date'] = df_matches['date'].astype(str)
df_matches['time'] = df_matches['time'].astype(str)
df_matches['date_time'] = df_matches['date'] + ' ' + df_matches['time']
df_matches['date_time'] = pd.to_datetime(df_matches['date_time'])

In [794]:
# Split score in df_matches by "-" to home_team_score and away_team_score 
df_matches[['home_team_score', 'away_team_score']] = df_matches['score'].str.split('–', expand=True)

In [795]:
df_matches.rename(columns={'game_id': 'MATCH_ID', 'date_time': 'DATE_TIME', 'home_team_id': 'HOME_TEAM_ID', 'away_team_id': 'AWAY_TEAM_ID',
                           'venue': 'STADIUM', 'week': 'GAMEWEEK', 'day': 'DAY','home_team_score': 'HOME_TEAM_SCORE', 
                           'away_team_score':'AWAY_TEAM_SCORE', 'home_xg':'HOME_TEAM_XG','away_xg':'AWAY_TEAM_XG',
                           'attendance':'ATTENDANCE', 'referee': 'REFEREE', 'ws_game_id':'WS_MATCH_ID'}, inplace=True)

In [796]:
df_matches.drop(columns=['date', 'time', 'home_team', 'away_team', 'score'], inplace=True)

In [797]:
df_matches['DATE_TIME'] = df_matches['DATE_TIME'].astype('string')

In [800]:
upsert_to_snowflake("MATCHES", df_matches, ['MATCH_ID'])

[02/03/24 00:32:58] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=799045;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=620325;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=831764;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=209232;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=21956;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=436389;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=767551;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=587798;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

[02/03/24 00:32:59] INFO     query execution done                                                     ]8;id=227211;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=236838;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

[02/03/24 00:33:02] INFO     query execution done                                                     ]8;id=310099;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=566388;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS MATCHES_TEMP]                               ]8;id=363707;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=531093;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=512629;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=503070;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=302462;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=601631;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 218 rows processed.


### Time to deal woth event data

In [398]:
ws_event_spadl = ws.read_events(output_fmt='spadl')

[02/02/24 22:20:36] INFO     Scraping game schedule from                                           ]8;id=173735;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=126809;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#427\427]8;;\
                             https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/9618/Engl                 
                             and-Premier-League                                                                    

                    INFO     Scraping game schedule for Thursday, Feb 1 2024                       ]8;id=632441;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=233473;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

                    INFO     Scraping game schedule for Saturday, Feb 3 2024                       ]8;id=746935;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=973822;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

[02/02/24 22:20:37] INFO     Scraping game schedule for Sunday, Feb 4 2024                         ]8;id=263133;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=184000;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

                    INFO     Scraping game schedule for Monday, Feb 5 2024                         ]8;id=783100;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=574223;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

...

                    INFO     [256/259] Retrieving game with id=1729271                             ]8;id=52683;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=908304;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#693\693]8;;\

                    INFO     [257/259] Retrieving game with id=1729307                             ]8;id=207979;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=952556;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#693\693]8;;\

                    INFO     [258/259] Retrieving game with id=1729316                             ]8;id=522077;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=877165;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#693\693]8;;\

[02/02/24 22:28:36] WARNING  No events found for game 1729316                                      ]8;id=670283;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=990566;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#757\757]8;;\

                    INFO     [259/259] Retrieving game with id=1729240                             ]8;id=507165;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=479349;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#693\693]8;;\

In [399]:
ws_event_spadl.loc[ws_event_spadl['team'] == 'Manchester United', 'team'] = 'Manchester Utd'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Newcastle United', 'team'] = 'Newcastle Utd'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Newcastle', 'team'] = 'Newcastle Utd'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Tottenham Hotspur', 'team'] = 'Tottenham'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Wolverhampton Wanderers', 'team'] = 'Wolves'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Wolverhampton', 'team'] = 'Wolves'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Nottingham Forest', 'team'] = "Nott'ham Forest"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Leicester', 'team'] = "Leicester City"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Leeds', 'team'] = "Leeds United"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Sheffield United', 'team'] = "Sheffield Utd"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Sheff Utd', 'team'] = "Sheffield Utd"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Luton', 'team'] = "Luton Town"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Norwich', 'team'] = "Norwich City"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Man City', 'team'] = "Manchester City"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Man Utd', 'team'] = "Manchester Utd"

In [400]:
cur.execute('SELECT * FROM TEAMS')
team_rows = cur.fetchall()
column_names = [desc[0] for desc in cur.description]
curr_team_names = pd.DataFrame(team_rows, columns=column_names)

[02/02/24 22:33:21] INFO     query: [SELECT * FROM TEAMS]                                             ]8;id=876119;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=996810;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=10028;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=825949;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 25                                     ]8;id=596402;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=802473;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [401]:
ws_event_spadl = ws_event_spadl.merge(spadl.actiontypes_df(), how="left").merge(spadl.bodyparts_df(), how="left")

In [402]:
ws_event_spadl.rename({'game_id':'WS_MATCH_ID', 'original_event_id':'ORIGINAL_EVENT_ID','period_id':'PERIOD_ID',
                       'time_seconds':'TIME_SECONDS', 'team_id':'TEAM_WS_ID', 'player_id':'PLAYER_WS_ID', 'start_x':'START_X',
                        'end_x':'END_X', 'start_y':'START_Y', 'end_y':'END_Y','type_name':'TYPE_NAME',
                        'bodypart_name':'BODYPART_NAME','result_id':'RESULT_ID', 'action_id':'ACTION_ID'}, axis=1, inplace=True)

In [403]:
ws_event_spadl = ws_event_spadl.merge(df_matches[['MATCH_ID', 'WS_MATCH_ID']], on='WS_MATCH_ID')

In [ ]:
ws_event_spadl = ws_event_spadl.merge(curr_team_names[['TEAM_FBREF_ID','TEAM_WS_ID']], on='TEAM_WS_ID')

In [ ]:
ws_event_spadl.rename({'TEAM_FBREF_ID':'team_id','START_X':'start_x','START_Y':'start_y','END_X':'end_x','END_Y':'end_y'}, axis=1,
                      inplace=True)

In [404]:
for index, row in df_matches.iterrows():
    inds_to_replace = ws_event_spadl[ws_event_spadl['MATCH_ID'] == row['MATCH_ID']].index
    ws_event_spadl.loc[inds_to_replace] = spadl.play_left_to_right(ws_event_spadl.loc[inds_to_replace], row['HOME_TEAM_ID'])

In [407]:
ws_event_spadl['NEW_TIME_SECONDS'] = ws_event_spadl['TIME_SECONDS']

In [408]:
ws_event_spadl_period1 = ws_event_spadl[ws_event_spadl['PERIOD_ID'] == 1]
ws_event_spadl_period2 = ws_event_spadl[ws_event_spadl['PERIOD_ID'] == 2]

In [409]:
start_time_2nd_half = ws_event_spadl_period2.groupby(['MATCH_ID', 'PERIOD_ID'])['NEW_TIME_SECONDS'].min().reset_index()

In [410]:
for index, row in tqdm(start_time_2nd_half.iterrows()):
    time_to_remove = row['NEW_TIME_SECONDS']
    ws_event_spadl.loc[(ws_event_spadl['MATCH_ID'] == row['MATCH_ID']) & (ws_event_spadl['PERIOD_ID'] == 2), 'NEW_TIME_SECONDS'] -= time_to_remove

218it [00:04, 46.68it/s]


In [331]:
# upsert_to_snowflake("EVENTS_SPADL", 
#                     ws_event_spadl.drop(['type_id', 'player', 'team', 'WS_MATCH_ID', 'TEAM_WS_ID','bodypart_id'],axis=1), 
#                     ['MATCH_ID','ACTION_ID'])

[02/02/24 20:28:19] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=220580;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=823186;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=326503;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=620325;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=525019;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=253602;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

[02/02/24 20:28:21] INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=70562;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=649753;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

[02/02/24 20:28:22] INFO     query execution done                                                     ]8;id=239429;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=412096;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

[02/02/24 20:28:35] INFO     query execution done                                                     ]8;id=529091;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=365548;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS EVENTS_SPADL_TEMP]                          ]8;id=135739;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=795674;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=153402;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=662368;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=209380;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=509791;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 338879 rows processed.


In [361]:
# cur.execute('SELECT * FROM EVENTS_SPADL')
# events_rows = cur.fetchall()
# column_names = [desc[0] for desc in cur.description]
# ws_event_spadl = pd.DataFrame(events_rows, columns=column_names)

[02/02/24 22:01:09] INFO     query: [SELECT * FROM EVENTS_SPADL]                                      ]8;id=633896;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=709349;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

[02/02/24 22:01:21] INFO     query execution done                                                     ]8;id=639737;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=898873;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 812                                    ]8;id=255347;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=97657;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

### Update the "Players" table in SNowflake

In [801]:
## Sort out "Players" "Table
ws_players.rename({'team_id':'TEAM_WS_ID', 'game_id':'WS_MATCH_ID'}, axis=1, inplace=True)

In [365]:
ws_players = ws_players.merge(curr_team_names, on="TEAM_WS_ID", how="left")

In [366]:
ws_players = ws_players.sort_values(by=["WS_MATCH_ID","TEAM_NAME","jersey_number"])

In [802]:
fbref_lineups.merge(curr_team_names[['TEAM_NAME','TEAM_FBREF_ID','TEAM_WS_ID']], on='TEAM_NAME')

,jersey_number,player,TEAM_NAME,is_starter,position,minutes_played,TEAM_FBREF_ID,TEAM_WS_ID
0,1,James Trafford,Burnley,True,GK,90,943e8050,184
1,2,Dara O'Shea,Burnley,True,CB,90,943e8050,184
2,5,Jordan Beyer,Burnley,True,CB,73,943e8050,184
3,14,Connor Roberts,Burnley,True,RB,90,943e8050,184
4,16,Sander Berge,Burnley,True,CM,89,943e8050,184
...,...,...,...,...,...,...,...,...
8737,17,Hugo Bueno,Wolves,False,<NA>,0,8cec06e1,161
8738,21,Pablo Sarabia,Wolves,False,"RW,AM",28,8cec06e1,161
8739,32,Joseph Hodge,Wolves,False,<NA>,0,8cec06e1,161
8740,62,Tawanda Chirewa,Wolves,False,<NA>,0,8cec06e1,161


In [803]:
fbref_lineups = fbref_lineups.reset_index().merge(fbref_schedule.reset_index()[['game', 'game_id',
                                                                                'ws_game_id']], how='right', on='game')

In [804]:
fbref_lineups = fbref_lineups.rename({'ws_game_id':'WS_MATCH_ID'}, axis=1)

In [805]:
fbref_lineups = fbref_lineups.merge(curr_team_names[['TEAM_NAME','TEAM_FBREF_ID','TEAM_WS_ID']], on='TEAM_NAME')

In [806]:
ws_players.rename({'player_id':'WS_player_id', 'player_name':'WS_player_name','minutes_played':'WS_minutes_played',
                   'starting_position':'WS_starting_position', 'is_starter':'WS_is_starter'}, axis=1, inplace=True)

In [807]:
fbref_lineups.rename({'minutes_played':'FBREF_minutes_played', 'game_id':'MATCH_ID','position':'FBREF_position',
                      'is_starter':'FBREF_is_starter'}, axis=1, inplace=True)

In [808]:
fbref_lineups = fbref_lineups.merge(ws_players, on=["TEAM_FBREF_ID", "TEAM_NAME", "WS_MATCH_ID",
                                                    "TEAM_WS_ID", "jersey_number"])

In [809]:
df_players = fbref_lineups[['WS_player_id', 'player', 'WS_player_name']].drop_duplicates()

In [810]:
df_players.rename({'WS_player_id':'PLAYER_WS_ID', 'player':'PLAYER_FBREF_NAME', 'WS_player_name':'PLAYER_WS_NAME'}, 
                  axis=1, inplace=True)

In [390]:
upsert_to_snowflake("PLAYERS", df_players, ['PLAYER_WS_ID'])

C:\Users\ksbha\AppData\Local\Temp\ipykernel_6388\138046206.py:6: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, nchunks, nrows, _ = write_pandas(conn, dataframe, temp_table_name, auto_create_table=True)


[02/02/24 22:10:48] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=551273;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=715872;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=497680;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=818713;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=887088;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=831396;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=450488;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=883711;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

...

[02/02/24 22:10:53] INFO     query execution done                                                     ]8;id=462751;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=128207;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS PLAYERS_TEMP]                               ]8;id=442133;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=750101;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=623754;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=99680;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=688954;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=999197;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 655 rows processed.


Player-match combos now

In [818]:
df_player_match = fbref_lineups[['WS_player_id', 'player','MATCH_ID', 'TEAM_FBREF_ID', 'WS_starting_position', 
                                 'FBREF_position','jersey_number','FBREF_minutes_played','FBREF_is_starter','WS_is_starter',
                                 'WS_minutes_played']]

In [819]:
df_player_match = df_player_match.rename({'WS_player_id':'PLAYER_WS_ID', 'player':'PLAYER_FBREF_NAME', 'WS_starting_position':'STARTING_POSITION_WS',
                        'FBREF_position':'POSITION_FBREF', 'jersey_number':'JERSEY_NUMBER', 'FBREF_minutes_played':'MINUTES_PLAYED_FBREF',
                        'FBREF_is_starter':'IS_STARTER_FBREF','WS_is_starter':'IS_STARTER_WS','WS_minutes_played':'MINUTES_PLAYED_WS'}, axis=1)

In [820]:
df_player_match.isna().sum()

PLAYER_WS_ID               0
PLAYER_FBREF_NAME          0
MATCH_ID                   0
TEAM_FBREF_ID              0
STARTING_POSITION_WS       0
POSITION_FBREF          2198
JERSEY_NUMBER              0
MINUTES_PLAYED_FBREF       0
IS_STARTER_FBREF           0
IS_STARTER_WS              0
MINUTES_PLAYED_WS          0
dtype: int64

In [821]:
upsert_to_snowflake("PLAYER_MATCH", df_player_match, ['PLAYER_WS_ID', 'MATCH_ID'])

[02/03/24 00:38:20] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=451594;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=168593;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=732145;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=260657;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=617709;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=864085;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=27623;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=457023;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

[02/03/24 00:38:21] INFO     query execution done                                                     ]8;id=526847;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=932582;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

[02/03/24 00:38:24] INFO     query execution done                                                     ]8;id=672768;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=258932;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS PLAYER_MATCH_TEMP]                          ]8;id=80893;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=74112;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=783738;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=738793;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=265159;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=883507;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 8702 rows processed.


### Add Shot Event Data

In [823]:
fbref_shot_events = fbref.read_shot_events()

[02/03/24 00:43:07] INFO     [1/219] Retrieving game with id=3a6836b4                                 ]8;id=237297;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=635600;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

[02/03/24 00:43:10] INFO     [2/219] Retrieving game with id=26a7f90c                                 ]8;id=469228;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=72253;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

[02/03/24 00:43:14] INFO     [3/219] Retrieving game with id=d6bbf293                                 ]8;id=643204;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=518834;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

[02/03/24 00:43:17] INFO     [4/219] Retrieving game with id=56a137f7                                 ]8;id=920016;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=980953;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

[02/03/24 00:43:21] INFO     [5/219] Retrieving game with id=15addfc7                                 ]8;id=123332;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=508242;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

...

[02/03/24 00:55:18] INFO     [215/219] Retrieving game with id=1836d9bd                               ]8;id=459674;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=346321;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

[02/03/24 00:55:21] INFO     [216/219] Retrieving game with id=d10aaae8                               ]8;id=207259;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=74644;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

[02/03/24 00:55:24] INFO     [217/219] Retrieving game with id=79ffec97                               ]8;id=506982;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=265287;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

[02/03/24 00:55:28] INFO     [218/219] Retrieving game with id=16dc9951                               ]8;id=748905;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=301552;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

[02/03/24 00:55:31] INFO     [219/219] Retrieving game with id=457203bc                               ]8;id=223289;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=480606;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

In [824]:
fbref_shot_events

minute  \
                                                                      
league             season game                                        
ENG-Premier League 2324   2023-08-11 Burnley-Manchester City      4   
                          2023-08-11 Burnley-Manchester City      6   
                          2023-08-11 Burnley-Manchester City     11   
                          2023-08-11 Burnley-Manchester City     15   
                          2023-08-11 Burnley-Manchester City     18   
...                                                             ...   
                          2024-02-01 Wolves-Manchester Utd     90+3   
                          2024-02-01 Wolves-Manchester Utd     90+4   
                          2024-02-01 Wolves-Manchester Utd     90+5   
                          2024-02-01 Wolves-Manchester Utd     90+7   
                          2024-02-01 Wolves-Manchester Utd     90+8   

                                                                       player  \
                                                                                
league             season game                                                  
ENG-Premier League 2324   2023-08-11 Burnley-Manchester City   Erling Haaland   
                          2023-08-11 Burnley-Manchester City  Kevin De Bruyne   
                          2023-08-11 Burnley-Manchester City   Erling Haaland   
                          2023-08-11 Burnley-Manchester City    Luca Koleosho   
                          2023-08-11 Burnley-Manchester City     Zeki Amdouni   
...                                                                       ...   
                          2024-02-01 Wolves-Manchester Utd             Antony   
                          2024-02-01 Wolves-Manchester Utd    Scott McTominay   
                          2024-02-01 Wolves-Manchester Utd         Pedro Neto   
                          2024-02-01 Wolves-Manchester Utd      Kobbie Mainoo   
                          2024-02-01 Wolves-Manchester Utd      Matheus Cunha   

                                                                         team  \
                                                                                
league             season game                                                  
ENG-Premier League 2324   2023-08-11 Burnley-Manchester City  Manchester City   
                          2023-08-11 Burnley-Manchester City  Manchester City   
                          2023-08-11 Burnley-Manchester City  Manchester City   
                          2023-08-11 Burnley-Manchester City          Burnley   
                          2023-08-11 Burnley-Manchester City          Burnley   
...                                                                       ...   
                          2024-02-01 Wolves-Manchester Utd     Manchester Utd   
                          2024-02-01 Wolves-Manchester Utd     Manchester Utd   
                          2024-02-01 Wolves-Manchester Utd             Wolves   
                          2024-02-01 Wolves-Manchester Utd     Manchester Utd   
                          2024-02-01 Wolves-Manchester Utd             Wolves   

                                                                xG  PSxG  \
                                                                           
league             season game                                             
ENG-Premier League 2324   2023-08-11 Burnley-Manchester City  0.17  0.62   
                          2023-08-11 Burnley-Manchester City  0.02  <NA>   
                          2023-08-11 Burnley-Manchester City  0.21  <NA>   
                          2023-08-11 Burnley-Manchester City  0.08  <NA>   
                          2023-08-11 Burnley-Manchester City  0.03  0.02   
...                                                            ...   ...   
                          2024-02-01 Wolves-Manchester Utd    0.06  <NA>   
                          2024-02-01

In [825]:
fbref_shot_events = fbref_shot_events.rename_axis(index={'season': 'SEASON', 'league': 'COMPETITION'})

In [826]:
fbref_shot_events = make_season_integer(fbref_shot_events)

In [827]:
ws_event_spadl['TIME_SECONDS'] = ws_event_spadl['TIME_SECONDS'].astype(int)
ws_event_spadl['PLAYER_WS_ID'] = ws_event_spadl['PLAYER_WS_ID'].astype(int)

In [828]:
ws_shots = ws_event_spadl[(ws_event_spadl['TYPE_NAME'] == 'shot') | (ws_event_spadl['TYPE_NAME'] == 'shot_freekick') | \
               (ws_event_spadl['TYPE_NAME'] == 'shot_penalty')]

In [829]:
fbref_shot_events.reset_index(inplace=True)

In [830]:
fbref_shot_events.columns = fbref_shot_events.columns.get_level_values(0) + fbref_shot_events.columns.get_level_values(1)

In [831]:
fbref_shot_events = fbref_shot_events.merge(fbref_schedule[['COMPETITION','SEASON','game','game_id','ws_game_id']], 
                        on=['COMPETITION','SEASON','game'])

In [832]:
fbref_shot_events.rename({'player':'PLAYER_FBREF_NAME'}, axis=1, inplace=True)

In [833]:
fbref_shot_events['PLAYER_FBREF_NAME'] = fbref_shot_events['PLAYER_FBREF_NAME'].str.replace(' \(pen\)', '', regex=True)

In [834]:
fbref_shot_events = fbref_shot_events.merge(df_players, on='PLAYER_FBREF_NAME', how="left")

In [835]:
fbref_shot_events.rename({'game_id':'MATCH_ID'}, axis=1, inplace=True)

In [836]:
# ws_shots = ws_event_spadl[(ws_event_spadl['TYPE_NAME'] == 'shot') | (ws_event_spadl['TYPE_NAME'] == 'shot_freekick') | \
#                (ws_event_spadl['TYPE_NAME'] == 'shot_penalty')]

In [837]:
ws_dropped_shots = ws_shots[ws_shots['PLAYER_WS_ID'] == 444304]
ws_shots = ws_shots[ws_shots['PLAYER_WS_ID'] != 444304]

In [857]:
## Rogue Joel Veltman shot should be dropped
ws_shots = ws_shots.drop(ws_shots[(ws_shots['PLAYER_WS_ID'] == 110290) & (ws_shots['MATCH_ID'] == '36be5cee')].index)

In [858]:
for game in ws_shots['MATCH_ID'].unique():
    if len(ws_shots.loc[ws_shots.MATCH_ID == game]) != len(fbref_shot_events.loc[fbref_shot_events.MATCH_ID == game]):
        print(game)

In [859]:
# fbref_shot_events[['new_minute', 'added_minute']] = fbref_shot_events['minute'].apply(
#     lambda x: pd.Series(str(x).split('+') if '+' in str(x) else [x, 0])
# )

In [860]:
# fbref_shot_events['new_minute'] = fbref_shot_events['new_minute'].astype(int)
# fbref_shot_events['added_minute'] = fbref_shot_events['added_minute'].astype(int)

In [861]:
# fbref_shot_events = fbref_shot_events.sort_values(by=['MATCH_ID', 'new_minute', 'added_minute', 'PLAYER_WS_ID']).reset_index(drop=True)

In [862]:
ws_shots = ws_shots.sort_values(by=['MATCH_ID', 'PERIOD_ID','ACTION_ID']).reset_index(drop=True)

In [863]:
ws_shots['xG'] = np.nan
ws_shots['PSxG'] = np.nan
ws_shots['outcome'] = np.nan
ws_shots['distance'] = np.nan
ws_shots['body_part'] = np.nan
ws_shots['notes'] = np.nan
ws_shots['SCA 1player'] = np.nan
ws_shots['SCA 1event'] = np.nan
ws_shots['SCA 2player'] = np.nan
ws_shots['SCA 2event'] = np.nan
ws_shots['FBREF_PLAYER_WS_ID'] = np.nan
ws_shots['PLAYER_WS_NAME'] = np.nan

In [864]:
fbref_shot_events.rename({'PLAYER_WS_ID':'FBREF_PLAYER_WS_ID'}, axis=1, inplace=True)

In [865]:
cols_of_interest = ['xG', 'PSxG', 'outcome', 'distance', 'body_part', 'notes', 'SCA 1player', 'SCA 1event', 
                    'SCA 2player', 'SCA 2event', 'FBREF_PLAYER_WS_ID', 'PLAYER_WS_NAME']
for game in ws_shots['MATCH_ID'].unique():
    df_hold = fbref_shot_events[fbref_shot_events['MATCH_ID'] == game]
    ind_list = ws_shots[ws_shots['MATCH_ID'] == game].index
    ws_shots.loc[ind_list, cols_of_interest] = np.array(df_hold[cols_of_interest])

C:\Users\ksbha\AppData\Local\Temp\ipykernel_6388\3612877618.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[<NA>, <NA>, 0.39, <NA>, <NA>, <NA>, <NA>, 0.25, 0.78, <NA>, <NA>, 0.05, 0.04, <NA>, 0.12, <NA>, 0.21, <NA>, <NA>, <NA>, <NA>, 0.98, <NA>, 0.08, <NA>, <NA>, <NA>, <NA>, <NA>, 0.03, <NA>, 0.94, 0.85]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ws_shots.loc[ind_list, cols_of_interest] = np.array(df_hold[cols_of_interest])
C:\Users\ksbha\AppData\Local\Temp\ipykernel_6388\3612877618.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Off Target', 'Off Target', 'Goal', 'Off Target', 'Blocked', 'Blocked', 'Off Target', 'Saved', 'Goal', 'Blocked', 'Off Target', 'Saved', 'Saved', 'Off Target', 'Saved', 'Blocked', 'Saved', 'Off Target', 'Blocked', 'Blocked', 'Blocked', 'Goal', 'Blocked', 'Saved

In [866]:
ws_shots['FBREF_PLAYER_WS_ID'] = ws_shots['FBREF_PLAYER_WS_ID'].astype(int)

In [871]:
mismatched_inds = ws_shots[ws_shots['PLAYER_WS_ID'] != ws_shots['FBREF_PLAYER_WS_ID']].index

In [872]:
len(mismatched_inds)

0

In [870]:
for i in range(len(mismatched_inds) // 2):
    val1 = ws_shots.loc[mismatched_inds, cols_of_interest].iloc[2*i]
    val2 = ws_shots.loc[mismatched_inds, cols_of_interest].iloc[(2*i)+1]

    ws_shots.loc[val1.name, cols_of_interest] = np.array(val2)
    ws_shots.loc[val2.name, cols_of_interest] = np.array(val1)

In [873]:
#### Remove this part later becasue it's just for Yarmolyuik

df_shots = ws_shots[['MATCH_ID','ACTION_ID','xG', 'PSxG', 'outcome', 'distance', 
          'body_part', 'notes', 'SCA 1player', 'SCA 1event', 'SCA 2player', 'SCA 2event']]

ws_dropped_shots[[cols_of_interest]] = np.nan

rows_to_add = [
    [0.04,np.nan,'Blocked',16,'Right Foot',np.nan,np.nan,np.nan,np.nan,np.nan,444304,'Yehor Yarmoliuk'],
[0.03,0.12,'Saved',27,'Right Foot',np.nan,np.nan,np.nan,np.nan,np.nan,444304,'Yehor Yarmoliuk'],
[0.05,np.nan,'Off Target',21,'Right Foot',np.nan,np.nan,np.nan,np.nan,np.nan,444304,'Yehor Yarmoliuk'],
[0.49,0.25,'Saved',11,'Right Foot',np.nan,np.nan,np.nan,np.nan,np.nan,444304,'Yehor Yarmoliuk'],
[0.09,np.nan,'Off Target',8,'Right Foot',np.nan,np.nan,np.nan,np.nan,np.nan,444304,'Yehor Yarmoliuk'],
[0.02,0.02,'Saved',19,'Right Foot',np.nan,np.nan,np.nan,np.nan,np.nan,444304,'Yehor Yarmoliuk']
]

ws_dropped_shots[cols_of_interest] = np.array(rows_to_add)

In [874]:
df_shots = pd.concat([df_shots, ws_dropped_shots[['MATCH_ID','ACTION_ID','xG', 'PSxG', 'outcome', 'distance', 
          'body_part', 'notes', 'SCA 1player', 'SCA 1event', 'SCA 2player', 'SCA 2event']]]).sort_index()

In [875]:
df_shots.replace({"Jarrell Quansah": "Jarell Quansah"}, inplace=True)
df_shots.replace({"Omotayo Adaramola": "Tayo Adaramola"}, inplace=True)

In [876]:
df_shots.rename({'xG':'XG', 'PSxG':'PSXG', 'outcome':'OUTCOME', 'distance':'DISTANCE', 'body_part':'BODY_PART',
                 'notes':'NOTES', 'SCA 1player':'SCA_1_PLAYER_FBREF_NAME', 'SCA 1event':'SCA_1_PLAYER_FBREF_EVENT',
                 'SCA 2player':'SCA_2_PLAYER_FBREF_NAME', 'SCA 2event':'SCA_2_PLAYER_FBREF_EVENT'}, axis=1, inplace=True)

In [877]:
df_players.rename({'PLAYER_FBREF_NAME':'SCA_1_PLAYER_FBREF_NAME'}, axis=1, inplace=True)
df_players.rename({'PLAYER_WS_ID':'SCA_1_PLAYER_FBREF_ID'}, axis=1, inplace=True)

In [878]:
df_shots = df_shots.merge(df_players[['SCA_1_PLAYER_FBREF_ID','SCA_1_PLAYER_FBREF_NAME']], on='SCA_1_PLAYER_FBREF_NAME', how='left')

In [879]:
df_players.rename({'SCA_1_PLAYER_FBREF_NAME':'SCA_2_PLAYER_FBREF_NAME'}, axis=1, inplace=True)
df_players.rename({'SCA_1_PLAYER_FBREF_ID':'SCA_2_PLAYER_FBREF_ID'}, axis=1, inplace=True)

In [880]:
df_shots = df_shots.merge(df_players[['SCA_2_PLAYER_FBREF_ID','SCA_2_PLAYER_FBREF_NAME']], on='SCA_2_PLAYER_FBREF_NAME', how='left')

In [881]:
df_shots = df_shots.drop(['SCA_1_PLAYER_FBREF_NAME', 'SCA_2_PLAYER_FBREF_NAME'], axis=1)

In [882]:
df_shots.rename({'SCA_1_PLAYER_FBREF_ID':'SCA_1_PLAYER_WS_ID', 'SCA_2_PLAYER_FBREF_ID':'SCA_2_PLAYER_WS_ID'}, 
                axis=1, inplace=True)

In [883]:
df_shots.replace({pd.NA: np.nan}, inplace=True)

In [884]:
df_shots['XG'] = df_shots['XG'].astype(float)
df_shots['PSXG'] = df_shots['PSXG'].astype(float)
df_shots['DISTANCE'] = df_shots['DISTANCE'].astype(int)

In [885]:
upsert_to_snowflake("SHOT_EVENTS", df_shots, ['MATCH_ID', 'ACTION_ID'])

[02/03/24 01:14:40] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=935723;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=700705;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=781214;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=739473;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=684983;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=932651;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

[02/03/24 01:14:41] INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=601130;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=842967;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=21743;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=106967;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

[02/03/24 01:14:45] INFO     query execution done                                                     ]8;id=433332;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=658127;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS SHOT_EVENTS_TEMP]                           ]8;id=94572;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=873122;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=224145;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=366631;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=778940;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=966408;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 5854 rows processed.
